In [ ]:
import pandas as pd
import plotly.express as px
import umap
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

In [2]:
# Read DF, separates training reactions and virtual reactions

df = pd.read_csv('../data/processed/CPA-virtual_database.csv')
df_training = df[df['Rxn_Type'] == 'Training']
df_training_rxns = df_training['rxn']
df_virtual = df[df['Rxn_Type'] == 'Virtual']
df_virtual = df_virtual[df_virtual['Catalyst_Ar_grp'] == 'L15']
df_virtual = df_virtual.reset_index(drop=True)

In [3]:
indexes = []
for rxn in df_virtual['rxn']:
    if rxn in df_training_rxns.values:
        index_names = df_virtual[ df_virtual['rxn'] == rxn].index
        indexes.append(index_names)

In [4]:
## Initial Processing ##

vt = VarianceThreshold()

parameters = df_virtual.loc[:,['iminium_nNH', 'iminium_iNH', 'iminium_N', 'iminium_H', 'iminium_C', 
                    'iminium_SubL', 'iminium_SubS', 'iminium_PG', 'iminium_PGL', 'iminium_PGB1', 
                    'iminium_PGB5', 'iminium_SL', 'iminium_SB1', 'iminium_SB5', 'iminium_LL', 'iminium_LB1', 
                    'iminium_LB5', 'iminium_HOMO', 'iminium_LUMO', 'iminium_Polarizability', 'iminium_EnergyDiff',
                    'nuc_H', 'nuc_X', 'nuc_Nu', 'nuc_nXH', 'nuc_iXH', 'nuc_HOMO', 'nuc_LUMO', 'nuc_Polarizability', 
                    'nuc_L', 'nuc_B1', 'nuc_B5', 'nuc_bondDistanceHX', 'nuc_H_X_Nu', 'nuc_H_X_CNu'
                    ]] ## Set parameters to desired columns ##
thres = vt.fit(parameters)
parameters = thres.transform(parameters)

scaled_data = StandardScaler().fit_transform(parameters)

In [5]:
## UMAP for Visualisation ##

reducer_vis = umap.UMAP(
    random_state=25,
    min_dist=0,  # Default is 0.1
    n_neighbors=len(scaled_data)/2,  # Default is 15
    n_components=2)

umap_2d = reducer_vis.fit_transform(scaled_data)


In [7]:
indx = pd.DataFrame(indexes)
df_virtual.iloc[indx[0],1] = 'Original'

fig = px.scatter(x=umap_2d[:,0], y=umap_2d[:,1], hover_data=[],
                labels={'x':'UMAP1', 'y':'UMAP2'},
                width=980, height=720,
                color_discrete_sequence=[px.colors.qualitative.T10[0], px.colors.qualitative.T10[2]],
                template='simple_white',
                color=df_virtual.Rxn_Type)

fig.update_traces(marker=dict(size=17,
                              line=dict(width=2,
                                        color='Black')),
                  selector=dict(mode='markers'))

fig.update_layout(
    legend=dict(
        yanchor="top",
        y=.99,
        xanchor="left",
        x=0.01,
        orientation='h',
        bordercolor="Black",
        borderwidth = 0
    ),
    legend_title_text=''
)

fig.update_xaxes(mirror=True)
fig.update_yaxes(mirror=True)


fig.update_layout(
    font_size=20,
    font_family="Arial"
)

fig.update_layout(showlegend=True)

fig.show()
